## 08 파이썬 추천 시스템 패키지 - Surprise

In [1]:
# 추천 시스템은 상업적으로 가치가 크기 때문에 별도의 패키지로 제공되면 매우 활용도가 높을 것이다.
# 이번에는 파이썬 기반의 추천 시스템 구축을 위한 전용 패키지 중의 하나인 Surprise를 이용해보자.

In [2]:
import surprise

print(surprise.__version__)

1.1.3


### Surprise를 이용한 추천 시스템 구축 

In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
data = Dataset.load_builtin('ml-100k')

# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/adam/.surprise_data/ml-100k


In [5]:
# Surprise에 사용자-아이템 평점 데이터를 적용할 때 주의해야 할 점은 무비렌즈 사이트에서 내려받은 데이터 파일과 동일하게
# 로우 레벨의 사용자-아이템 평점 데이터를 그대로 적용해야 한다는 것이다.

# Surprise는 자체적으로 로우 레벨의 데이터를 컬럼 레벨의 데이터로 변경하므로 원본인 로우 레벨의 사용자-아이템 데이터를
# 데이터 세트로 적용해야 한다.

In [6]:
algo = SVD()
algo.fit(trainset)

In [7]:
# 학습된 추천 알고리즘을 기반으로 테스트 데이터 세트에 대해 추천을 수행한다.

predictions = algo.test(testset)
print('prediction type:', type(predictions), 'size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type: <class 'list'> size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.8084581197757266, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.8728735821531353, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.166916445426095, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.5622690607201055, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.4222043943781792, details={'was_impossible': False})]

In [8]:
# 'was_impossible'이 True이면 예측값을 생성할 수 없는 데이터라는 의미이다.

In [9]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.8084581197757266),
 ('882', '291', 3.8728735821531353),
 ('535', '507', 4.166916445426095)]

In [10]:
# 이번에는 Surprise의 다른 추천 예측 메서드인 predict()를 이용해 추천 예측을 해보자.

# 사용자 아이디, 아이템 아이디는 문자여롤 입력해야 함.
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.30   {'was_impossible': False}


In [11]:
# 결과처럼 predict()는 개별 사용자와 아이템 정보를 입력하면 추천 예측 평점을 est로 반환한다.

# 테스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점과의 차이를 평가해보자.
# Surprise의 accuracy 모듈은 RMSE, MSE 등의 방법으로 추천 시스템의 성능 평가 정보를 제공한다.

In [14]:
accuracy.rmse(predictions)

RMSE: 0.9492


0.9492082894616816

In [15]:
# 이처럼 Surprise 패키지를 이용하면 쉽게 추천 시스템을 구현할 수 있다.

### Surprise 주요 모듈 소개 

#### Dataset 

In [16]:
# Surprise는 user_id(사용자 아이디), item_id(아이템 아이디), rating(평점) 데이터가 로우 레벨로 된 데이터 세트에만 적용할 수 있다.
# 일반 데이터 파일이나 판다스 DataFrame에서도 로딩을 할 수 있지만, 데이터 세트의 컬럼 순서가 사용자 아이디, 아이템 아이디, 평점 순으로
# 반드시 되어 있어야 한다. 4번째 컬럼부터는 로딩 자체를 수행하지 않는다.

#### OS 파엘 데이터를 Surprise 데이터 세트로 로딩

In [17]:
# Surprise에 OS 파일을 로딩할 때 주의할 점은 로딩 되는 데이터 파일에 컬럼명을 가지는 헤더 문자열이 있어서는 안된다는 것이다.
# 판다스 DataFramedml to_csv() 함수를 이용해 간단하게 이 컬럼의 헤더를 삭제하고 새로운 파일인 ratings_noh.csv로 저장한다.

In [18]:
import pandas as pd

ratings = pd.read_csv('/Users/adam/Data_Analytics/Python/Datasets/ml-latest-small/ratings.csv')

# ratings_noh.csv 파일로 언로드시 인덱스와 헤더를 모두 제거한 새로운 파일 생성.
ratings.to_csv('/Users/adam/Data_Analytics/Python/Datasets/ml-latest-small/ratings_noh.csv', index=False, header=False)

In [19]:
# 새롭게 생성된 ratings_noh.csv 파일은 tarings.csv 파일에서 헤더가 삭제된 파일이다.

In [21]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('/Users/adam/Data_Analytics/Python/Datasets/ml-latest-small/ratings_noh.csv', reader=reader)

In [22]:
# Surprise 데이터 세트는 기본적으로 무비렌즈 데이터 형식을 따르므로 무비렌즈 데이터 형식이 아닌 다른 OS 파일의 경우
# Reader 클래스를 먼저 설정해야 한다.

In [25]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시 마다 동일한 결과를 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### 판다스 DataFrame에서 Surprise 데이터 세트로 로딩

In [26]:
# 판다스 DataFrame 역시 사용자 아이디, 아이템 아이디, 평점 컬럼 순서를 지켜야 한다.
# DataFrame으로 로딩한 ratings에서 Surprise 데이터 세트로 로딩하려면
# Dataset.load_from_df(ratings[['userId','movieId','rating']].reader)와 같이 파라미터를 입력하면 된다.

In [30]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/Users/adam/Data_Analytics/Python/Datasets/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 한다.
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### 베이스라인 평점 

In [32]:
# 영화나 상품의 평가도 각 개인의 성향에 따라 같은 아이템이라도 평가가 달라질 수 있다.
# 싫은 소리를 별로 안하는 사람의 경우는 전반적으로 평가에 후한 경향이 있다.
# 반면에 다른 이를 생각해서라도 냉정한 평가를 해야 한다고 생각하는 사람도 있다.
# 이러한 개인의 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영하여 평점을 부과하는 것을 베이스라인 평점(Baseline Rating)이라고 한다.

# 보통 베이스라인 평점은 전체 평균 평점 + 사용자 편향 점수 + 아이템 평향 점수 공식으로 계산된다.
# * 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
# * 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
# * 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점

# 예) 모든 사용자의 평균 영화 평점: 3.5, 깐깐한 영화 매니아 사용자의 평균 평점: 3.0, 어벤저스 3편 평균 평점 4.2
# 3.5 + (3.0-3.50=0.5) + (4.2-3.5=0.7) = 3.7

### 교차 검증과 하이퍼 파라미터 튜닝 

In [36]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('/Users/adam/Data_Analytics/Python/Datasets/ml-latest-small/ratings.csv')
reader = Reader(rating_scale = (0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8731  0.8782  0.8642  0.8727  0.8784  0.8733  0.0052  
MAE (testset)     0.6705  0.6736  0.6666  0.6696  0.6771  0.6715  0.0036  
Fit time          0.60    0.59    0.60    0.60    0.61    0.60    0.01    
Test time         0.07    0.16    0.07    0.07    0.07    0.09    0.04    


{'test_rmse': array([0.87312823, 0.87823223, 0.86416387, 0.87272927, 0.87835783]),
 'test_mae': array([0.67053055, 0.67363746, 0.66659419, 0.66958531, 0.67705995]),
 'fit_time': (0.5976829528808594,
  0.5946669578552246,
  0.5977551937103271,
  0.5956947803497314,
  0.6097660064697266),
 'test_time': (0.06960487365722656,
  0.1621401309967041,
  0.07031488418579102,
  0.07067608833312988,
  0.07074689865112305)}

In [37]:
# cross_validate()는 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 보여준다.

In [38]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors':[50, 100, 200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8784168217002932
{'n_epochs': 20, 'n_factors': 50}


### Surprise를 이용한 개인화 영화 추천 시스템 구축

In [39]:
# Surprise를 이용해 잠재 요인 협업 필터링 기반의 개인화된 영화 추천을 구현해보자.

# 이번 예제에서는 ratings.csv 데이터를 학습 데이터와 테스트 데이터로 분리하지 않고 전체를 학습 데이터로 사용한다.

In [40]:
# 다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 이 처럼 오류가 발생한다.
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

In [41]:
# 데이터 세트 전체를 학습 데이터로 사용하려면 DatasetAutoFolds 클래스를 이용하면 된다.
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))

# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 형성.
data_folds = DatasetAutoFolds(ratings_file='/Users/adam/Data_Analytics/Python/Datasets/ml-latest-small/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성함.
trainset = data_folds.build_full_trainset()

In [42]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [43]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('/Users/adam/Data_Analytics/Python/Datasets/ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인.
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [44]:
# 학습된 SVD 객체에서 predict() 메서드 내에 userId와 movieId 값을 입력해주면 된다. (이 값은 모두 문자열 이어야 함)
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [45]:
# 추천 예측 평점은 est값으로 3.13이다.

In [46]:
# 이제 사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤에 예측 평점 순으로 영화를 추출해보자.

In [48]:
def get_unseen_surprise(ratings, movies, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성.
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:', len(seen_movies), '추천대상 영화수:', len(unseen_movies), \
        '전체 영화수:', len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [51]:
# 사용자 아이디 9번은 전체 9742개의 영화 중에서 46개만 평점을 매겼다.
# 따라서 추천 대상 영화는 9696개이며, 이 중 앞에서 학습된 추천 알고리즘 클래스인 SVD를 이용해 높은 예측 평점을 가진 순으로
# 영화를 추천 해보자.

def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수로 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n 개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in \
                       zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Ttop-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Ttop-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


In [ ]:
# 9번 아이디 사용자에게는 유주얼 세스벡트, 스타워즈, 펄프픽션 등이 주로 추천 되었다.
# 이처럼 Surprise 패키지는 복잡한 알고리즘을 직접 구현하지 않고도 쉽고 간결한 API를 이용해 파이썬 기반에서
# 추천 시스템을 구축할 수 있도록 해준다.